In [1]:
import squigglepy as sq
import numpy as np
import pandas as pd
from squigglepy.numbers import K, M, B
from chip_estimates_utils import (
    normalize_shares,
    compute_h100_equivalents,
    export_quarterly_by_version,
    print_cumulative_summary,
    estimate_chip_sales
)

sq.set_seed(42)
np.random.seed(42)
N_SAMPLES = 5000
H100_TOPS = 1979

In [2]:
# ======================
# Specs and data inputs
# ======================

# TPU specs: 8-bit TOPS and manufacturing costs
TPU_SPECS = {
    'v3':  {'tops': 123,  'cost': sq.to(940, 1400)},
    'v4':  {'tops': 275,  'cost': sq.to(1100, 1500)},
    'v5e': {'tops': 393,  'cost': sq.to(950, 1400)},
    'v5p': {'tops': 918,  'cost': sq.to(2300, 2900)},
    'v6e': {'tops': 1836, 'cost': sq.to(1600, 1900)},
    'v7':  {'tops': 4614, 'cost': sq.to(4600, 5500)},
}

# Broadcom margins (higher in FY23, lower afterward)
MARGIN_FY23 = sq.to(0.60, 0.75)
MARGIN = sq.to(0.50, 0.70)

# ==========================
# Quarterly data/estimates on TPU revenue by quarter and production mix
# To do: figure out a more friendly way to store this data, e.g. CSV or Google sheet import
# ==========================

# TPU revenue to Broadcom by quarter ($ billions, p5-p95 ranges)
TPU_REVENUE = {
    'Q1_FY23': sq.to(0.42, 0.53), 'Q2_FY23': sq.to(0.53, 0.66),
    'Q3_FY23': sq.to(0.53, 0.66), 'Q4_FY23': sq.to(0.79, 0.99),
    'Q1_FY24': sq.to(1.25, 1.40), 'Q2_FY24': sq.to(1.65, 1.90),
    'Q3_FY24': sq.to(1.85, 2.00), 'Q4_FY24': sq.to(2.10, 2.30),
    'Q1_FY25': sq.to(2.20, 2.50), 'Q2_FY25': sq.to(2.40, 2.80),
    'Q3_FY25': sq.to(2.90, 3.20), 'Q4_FY25': sq.to(3.60, 4.00),
}

# Production mix by quarter (p5-p95 ranges, will be normalized later)
PROD_MIX = {
    'Q1_FY23': {'v4': sq.to(0.70, 0.85), 'v3': sq.to(0.15, 0.30)},
    'Q2_FY23': {'v4': sq.to(0.90, 0.99), 'v3': sq.to(0.01, 0.10)},
    'Q3_FY23': {'v4': sq.to(0.87, 0.98), 'v5e': sq.to(0.01, 0.08)},
    'Q4_FY23': {'v4': sq.to(0.80, 0.92), 'v5e': sq.to(0.08, 0.20)},
    'Q1_FY24': {'v5e': sq.to(0.35, 0.60), 'v4': sq.to(0.35, 0.60), 'v5p': sq.to(0.02, 0.08)},
    'Q2_FY24': {'v5e': sq.to(0.65, 0.85), 'v5p': sq.to(0.15, 0.30)},
    'Q3_FY24': {'v5e': sq.to(0.30, 0.70), 'v5p': sq.to(0.25, 0.60), 'v6e': sq.to(0.01, 0.03)},
    'Q4_FY24': {'v5e': sq.to(0.30, 0.55), 'v5p': sq.to(0.30, 0.55), 'v6e': sq.to(0.01, 0.08)},
    'Q1_FY25': {'v5e': sq.to(0.15, 0.40), 'v5p': sq.to(0.15, 0.40), 'v6e': sq.to(0.10, 0.25)},
    'Q2_FY25': {'v5e': sq.to(0.20, 0.35), 'v5p': sq.to(0.20, 0.35), 'v6e': sq.to(0.30, 0.45)},
    'Q3_FY25': {'v5e': sq.to(0.10, 0.30), 'v5p': sq.to(0.10, 0.35), 'v6e': sq.to(0.35, 0.65), 'v7': sq.to(0.01, 0.10)},
    'Q4_FY25': {'v5e': sq.to(0.01, 0.25), 'v5p': sq.to(0.05, 0.25), 'v6e': sq.to(0.40, 0.80), 'v7': sq.to(0.10, 0.20)},
}

In [3]:
def get_price_distribution(version, is_fy23=False):
    """Get price distribution for a TPU version: cost / (1 - margin)."""
    margin = MARGIN_FY23 if is_fy23 else MARGIN
    return TPU_SPECS[version]['cost'] / (1 - margin)

# Pre-compute price distributions for each version and margin regime
PRICE_DIST_FY23 = {version: get_price_distribution(version, is_fy23=True) for version in TPU_SPECS}
PRICE_DIST = {version: get_price_distribution(version, is_fy23=False) for version in TPU_SPECS}

# Define sampling functions which get passed into estimate_chip_sales
def sample_revenue(quarter):
    # draw one sample from the quarter's distribution
    return (TPU_REVENUE[quarter] @ 1) * B

def sample_shares(quarter):
    mix = PROD_MIX[quarter]
    # draw one sample from each version's distribution
    raw_shares = {version: dist @ 1 for version, dist in mix.items()}
    return normalize_shares(raw_shares)

def sample_price(quarter, version):
    price_dists = PRICE_DIST_FY23 if 'FY23' in quarter else PRICE_DIST
    return price_dists[version] @ 1

## Quarter correlations

In [4]:

def estimate_chip_sales_correlated(quarters, versions, sample_revenue, sample_shares, sample_price, n_samples=5000):
    results = {quarter: {version: [] for version in versions} for quarter in quarters}

    # correlate price for now, we'll make this flexible later
    # this loop pre-samples prices, and then uses the same price sample in the loop by quarter
    price_samples = {version: [] for version in versions}

    for version in versions:
        for _ in range(n_samples):
            # to do: handle changing prices over time
            # in that case we want price distributions to be correlated across time... but not identical
            # could apply a deflation factor to the same distribution
            price_samples[version].append(sample_price(quarters[0], version))
    
    def sample_price_correlated(version, index):
        return price_samples[version][index]

    # to do, use price_samples in this loop

    for i in range(n_samples):
        for quarter in quarters:
            revenue = sample_revenue(quarter)
            shares = sample_shares(quarter)

            for version in versions:
                share = shares.get(version, 0)
                if share > 0:
                    price = sample_price_correlated(version, i) #sample_price(quarter, version)
                    chips = (revenue * share) / price
                else:
                    chips = 0
                results[quarter][version].append(chips)

    return results

    # return estimate_chip_sales(quarters, versions, sample_revenue, sample_shares, sample_price, n_samples=5000)


In [5]:
# super wide distribution as test
def sample_price_test(quarter, version):
    return sq.to(1000, 100000, lclip = 0) @ 1

test_results = estimate_chip_sales(
    quarters=list(TPU_REVENUE.keys()),
    versions=list(TPU_SPECS.keys()),
    sample_revenue=sample_revenue,
    sample_shares=sample_shares,
    sample_price=sample_price_test,
    n_samples=N_SAMPLES
)



In [6]:
test_results_corr = estimate_chip_sales_correlated(
    quarters=list(TPU_REVENUE.keys()),
    versions=list(TPU_SPECS.keys()),
    sample_revenue=sample_revenue,
    sample_shares=sample_shares,
    sample_price=sample_price_test,
    n_samples=N_SAMPLES
)

In [7]:
cumulative_1 = {version: np.zeros(N_SAMPLES) for version in TPU_SPECS}
for quarter in test_results:
    for version in TPU_SPECS:
        cumulative_1[version] += np.array(test_results[quarter][version])

print_cumulative_summary(cumulative_1, TPU_SPECS, "Cumulative TPU Production")

cumulative_corr = {version: np.zeros(N_SAMPLES) for version in TPU_SPECS}
for quarter in test_results_corr:
    for version in TPU_SPECS:
        cumulative_corr[version] += np.array(test_results_corr[quarter][version])

print_cumulative_summary(cumulative_corr, TPU_SPECS, "Cumulative TPU Production, correlated")


Cumulative TPU Production
Version           p5          p50          p95
---------------------------------------------
v3            2,346       15,336      115,874
v4          161,945      544,574    2,121,714
v5e         512,485    1,399,894    4,287,253
v5p         373,501    1,019,251    3,417,084
v6e         279,046      972,166    4,265,787
v7           13,915       90,005      656,549
---------------------------------------------
TOTAL     2,759,787    4,991,096   10,711,646

Cumulative TPU Production, correlated
Version           p5          p50          p95
---------------------------------------------
v3            1,178       11,902      115,904
v4           27,318      290,985    2,986,398
v5e          69,127      669,055    7,014,399
v5p          54,315      519,711    5,472,817
v6e          59,879      586,022    6,074,425
v7            6,974       72,098      745,345
---------------------------------------------
TOTAL     1,162,236    3,933,002   16,342,423


As expected, the correlated version of the simulation has much higher variance in cumulative totals, given the highly variable price distribution is correlated across quarters.

However, the implementation is not abstracted yet. 

Price is the main uncertainty we want to correlate across quarters. Uncertainty across revenue and chip mixes might also be correlated but seem less important.